In [1]:
import pandas as pd
import numpy as np

In [2]:
news = pd.read_csv('analyst_ratings_processed.csv')
news.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [3]:
news.columns

Index(['Unnamed: 0', 'title', 'date', 'stock'], dtype='object')

In [4]:
news = news[['title', 'date', 'stock']]
news.head()

,title,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [5]:
# Lets have a look on the sunique stocks we have

news['stock'].unique()

array(['A', 'AAMC', 'AAME', ..., 'ZU', 'ZUMZ', 'ZX'], dtype=object)

In [6]:
# Now let's see how many instances we have for each stock

max_count = news.groupby('stock').count().iloc[:, 0].max()
stock_name = news.groupby('stock').count().iloc[:, 0][news.groupby('stock').count().iloc[:, 0] == max_count].index[0]
stock_name


'MRK'

In [7]:
# lets extract the news for this stock
new_news = news[news['stock'] == stock_name].reset_index(drop = True)

new_news.head()

,title,date,stock
0,Shares of several healthcare companies are tra...,2020-06-11 10:22:00-04:00,MRK
1,Johnson & Johnson To Start Coronavirus Vaccine...,2020-06-11 00:16:00-04:00,MRK
2,The Daily Biotech Pulse: Keytruda Setback For ...,2020-06-10 07:30:00-04:00,MRK
3,Merck Announces That The Phase 3 KEYNOTE-361 T...,2020-06-09 16:13:00-04:00,MRK
4,"The Week Ahead In Biotech: Viela FDA Decision,...",2020-06-07 13:43:00-04:00,MRK


In [8]:
new_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3334 entries, 0 to 3333
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   3334 non-null   object
 1   date    3334 non-null   object
 2   stock   3334 non-null   object
dtypes: object(3)
memory usage: 78.3+ KB


In [9]:
new_news['date'] = pd.to_datetime(new_news['date'])
new_news['date'] = [item.date() for item in new_news['date']]
new_news.head()

,title,date,stock
0,Shares of several healthcare companies are tra...,2020-06-11,MRK
1,Johnson & Johnson To Start Coronavirus Vaccine...,2020-06-11,MRK
2,The Daily Biotech Pulse: Keytruda Setback For ...,2020-06-10,MRK
3,Merck Announces That The Phase 3 KEYNOTE-361 T...,2020-06-09,MRK
4,"The Week Ahead In Biotech: Viela FDA Decision,...",2020-06-07,MRK


In [10]:
new_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3334 entries, 0 to 3333
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   3334 non-null   object
 1   date    3334 non-null   object
 2   stock   3334 non-null   object
dtypes: object(3)
memory usage: 78.3+ KB


In [11]:
# First of all check the timestamp

start_date = new_news['date'].iloc[0]
end_date = new_news['date'].iloc[-1]

start_date, end_date

(datetime.date(2020, 6, 11), datetime.date(2009, 7, 27))

In [12]:
# Import Yahoo finance library

import yfinance as yf

# Get the data for the stock MRK
data = yf.download('MRK','2009-07-27','2020-06-11')

[*********************100%***********************]  1 of 1 completed


In [13]:
# Print 1st five rows of the data

data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-07-27,29.694656,29.751907,29.026718,29.360687,18.955818,21026758
2009-07-28,29.303434,29.494274,28.406488,28.645039,18.493776,23627265
2009-07-29,28.635496,28.759542,28.158398,28.501907,18.401369,16696736
2009-07-30,28.826336,29.551527,28.511450,28.568703,18.444498,24300290
2009-07-31,28.816793,29.179390,28.540075,28.635496,18.487617,16269362


In [14]:
# Function used to concatenate the news data while using group by function

def concat_data(x):
    return ' '.join(x)

In [15]:
# Create the aggregate news for different date values

news_agg = new_news.groupby('date', axis = 0)['title'].apply(concat_data).sort_index()

In [18]:
# Add a new column named as news in the data and assigned the concatenated news to this column

data['news'] = news_agg

In [19]:
data.isna().sum()

Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume          0
news         1289
dtype: int64

In [20]:
data.ffill(inplace = True)

data.head()

,Open,High,Low,Close,Adj Close,Volume,news
Date,,,,,,,
2009-07-27,29.694656,29.751907,29.026718,29.360687,18.955818,21026758,Wall Street News Alert: Stocks This Morning: ...
2009-07-28,29.303434,29.494274,28.406488,28.645039,18.493776,23627265,Wall Street News Alert: Stocks This Morning: ...
2009-07-29,28.635496,28.759542,28.158398,28.501907,18.401369,16696736,Wall Street News Alert: Stocks This Morning: ...
2009-07-30,28.826336,29.551527,28.511450,28.568703,18.444498,24300290,Wall Street News Alert: Stocks This Morning: ...
2009-07-31,28.816793,29.179390,28.540075,28.635496,18.487617,16269362,Wall Street News Alert: Stocks This Morning: ...


In [21]:
data.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
news         0
dtype: int64

In [22]:
# Create a new data set with only columns which we need

new_data = data[['Close', 'news']]

# Now since we have everything we need in a separate data frame it is time to free up some space by deleting the old dataframes

del data
del news
del new_news
del news_agg

In [23]:
new_data.head()

,Close,news
Date,,
2009-07-27,29.360687,Wall Street News Alert: Stocks This Morning: ...
2009-07-28,28.645039,Wall Street News Alert: Stocks This Morning: ...
2009-07-29,28.501907,Wall Street News Alert: Stocks This Morning: ...
2009-07-30,28.568703,Wall Street News Alert: Stocks This Morning: ...
2009-07-31,28.635496,Wall Street News Alert: Stocks This Morning: ...


In [24]:
# Import the seaborn and matplotlib libraries

import seaborn as sns
import matplotlib.pyplot as plt